# Summary 

author: Jeffrey Lin 
Date: 09/22/2025 

_This notebook identifies which LD blocks were signals for multiple proteins_
- checks for LD blocks associated with multiple proteins at different cv-r2 and cv-r thresholds

In [5]:
library(tidyverse)
library(patchwork) 
library(janitor)
library(readxl)

In [6]:
# Function Definitions 

# Function Name: find_ld_block_sharing
# Purpose: Determines how many proteins share the same significant ld_block 
# Input: dataframe containing ld block info, list of ld blocks of interest, column for significance threshold
# Output: Dataframe with rows corresponding to significat ld signals
find_ld_block_sharing <- function(data, ld_block, signal_col) {
    sig_results <- data %>% 
        filter(
            region_name %in% ld_block,
            !!sym(signal_col) == 1) %>%
            group_by(region_name) %>%
            arrange(desc(region_name)) 

    return(sig_results)
        
}

# Function Name: count_overlap
# Purpose: Sort LD blocks by number of proteins they are associated with
# Input: Dataframe with number of significant LD blocks, variable to group by 
# Ouput: Dataframe sorted by number of associations with proteins
count_overlap <- function(data, grouping) {
    data %>% 
        count(!!sym(grouping)) %>%
        arrange(desc(n)) %>%
        return()
    
}

In [46]:
# Read in Data on LD Blocks by Protein
ld_block_by_protein <- read_csv(
    file = "/gpfs/gibbs/project/dewan/jl4528/combined_regions.csv", 
    na = c("", " ", "NA", "na", ".")) %>%
    janitor::clean_names() %>% 
    suppressMessages()
                                
head(ld_block_by_protein)

# Read in Data on Proteins Predicted with Only Cis Variants
cis_only_predict <- read_xlsx(
    path = "/gpfs//gibbs/project/dewan/jl4528/466_cis-only_proteins.xlsx",
    na = c("", " ", "NA", "na", "." )
    ) %>% janitor::clean_names() %>%
    rename(avg_r2 = avg_r2, avg_r = avg_corr) %>%
    mutate(
        protein = trimws(protein)
    )
    

head(cis_only_predict)

# Read in LD Block summary data and handle duplicated rows
ld_block_summary_df <- read_xlsx(
    path = "/gpfs/gibbs/project/dewan/jl4528/LD_blocks_and_ST19.xlsx", 
    na = c("", " ", "NA", "na", "."),
    skip = 1,
    ) %>%  
    janitor::clean_names() %>% 
    mutate(
        protein = trimws(tolower(protein)),
    ) %>% 
    rename(
        cis_pqtls = p_qtl_component,
        trans_pqtls = x7,
        all_pqtls = x8
    ) %>% slice(-1) %>%
    mutate(across(where(is.character), ~ str_replace_all(.x, "-", "_")))

duplicated_proteins <- ld_block_summary_df %>% 
    count(protein) %>%
    filter(n != 1) %>% 
    pull(protein)

proteins_need_combining <- c("c19orf12", "c2orf69", "c7orf50", "c9orf40", "ntprobnp")
proteins_dup_copies <- setdiff(duplicated_proteins, proteins_need_combining)

combined_proteins <- ld_block_summary_df %>% 
    filter(protein %in% proteins_need_combining) %>%
    group_by(protein) %>%
    summarise(across(everything(), ~first(.x[!is.na(.x) & .x != ""])), .groups = "drop")

proteins_take_first <- ld_block_summary_df %>% 
    filter(protein %in% proteins_dup_copies) %>% 
    group_by(protein) %>% 
    slice(1) %>% 
    ungroup()

non_duplicate_proteins <- ld_block_summary_df %>% 
    filter(!(protein %in% c(proteins_need_combining, proteins_dup_copies))) 

ld_block_summary_df <- bind_rows(
    combined_proteins, 
    proteins_take_first, 
    non_duplicate_proteins
    ) %>% arrange(protein) 
    

head(ld_block_summary_df)

# Combining Datasets to create cumulative cvr2 and heritability estimate table
cv_results <- read_xlsx(
    path = "/gpfs/gibbs/project/dewan/jl4528/UKBBP_SM_Tables.xlsx", 
    na = c("", " ", "NA", "na", "."),
    skip = 1,
    sheet = "Table S1_FDR0.2_CV_method") %>%  
    janitor::clean_names() %>% 
    mutate(
        protein = trimws(tolower(protein))
    ) %>% arrange(protein) 

head(cv_results)


# Remove proteins with high missing rate 
proteins_to_remove <- setdiff(ld_block_by_protein$protein, cv_results$protein)

ld_block_by_protein <- ld_block_by_protein %>% 
    filter(!(protein %in% proteins_to_remove))

ld_block_summary_df <- ld_block_summary_df %>% 
    filter(!(protein %in% proteins_to_remove))

# Combine cv results with LD block summary data 
ld_block_summary_df <- left_join(ld_block_summary_df, cv_results, join_by(protein), relationship = "many-to-many")
head(ld_block_summary_df)

protein,chr,region_start,region_end,region_name,n_unfiltered_variants,n_filtered_variants,min_pval,signal_fdr0_1,signal_fdr0_2
<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
a1bg,01,100833965,102040534,01_100826405_102041016,238,238,0.0050532459,0,0
a1bg,01,102046569,102897158,01_102041016_102898745,170,170,0.0089146101,0,0
a1bg,01,102899784,103784956,01_102898745_103914211,144,143,0.0049031541,0,0
a1bg,01,103917590,106087842,01_103914211_106087842,330,329,0.0035578429,0,0
a1bg,01,756604,1892048,01_10583_1892607,419,392,0.0005231881,0,1
a1bg,01,106087842,108409080,01_106087842_108409665,462,460,0.0088391596,0,0


protein,best_method,avg_r2,sd_r2,avg_r
<chr>,<chr>,<dbl>,<dbl>,<dbl>
a1bg,bayes_r_weights,0.15902512,0.007180905,0.3986986
aamdc,lasso_weights,0.20415898,0.016673489,0.4515408
abo,lasso_weights,0.64641537,0.008525111,0.8039856
ace,susie_weights,0.07909728,0.010133860,0.2807842
ache,susie_weights,0.06227520,0.006097803,0.2493179
acp1,enet_weights,0.07662419,0.006555114,0.2765964


New names:
• `` -> `...7`
• `` -> `...8`


protein,number_cis_ld_blocks,number_trans_ld_blocks,number_total_ld_blocks,ukbppp_protein_id,cis_pqtls,trans_pqtls,all_pqtls,polygenic_component,total_heritability_th,proportion_of_th_explained_by_primary_cis_p_qtl_component,proportion_of_th_explained_by_primary_trans_p_qtl_component
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
a1bg,466,1,465,A1BG:P04217:OID30771:v1,9.6009252027164205E_2,1.50448618348897E_2,0.11105411386205299,0.1234,0.23445411,0.40950125,0.064169750
aamdc,391,3,388,AAMDC:Q9H7C9:OID30236:v1,0.22947367197313601,1.32634318179364E_3,0.23080001515492901,0.0774,0.30820002,0.74456087,0.004303514
aarsd1,391,2,389,AARSD1:Q9BTE6:OID21311:v1,1.1267267571340001E_2,3.4975055187944299E_3,1.47647730901344E_2,0.0523,0.06706477,0.16800575,0.052151157
abca2,444,2,442,ABCA2:Q9BZC7:OID30146:v1,1.8771451437145899E_3,0,1.8771451437145899E_3,0.0244,0.02627715,0.07143642,0.000000000
abhd14b,398,2,396,ABHD14B:Q96IU4:OID20921:v1,6.3093345880260401E_2,4.7071662658122399E_3,6.7800512146072703E_2,0.0635,0.13130051,0.48052627,0.035850327
abl1,403,1,402,ABL1:P00519:OID21280:v1,0,3.2398549045864998E_3,3.2398549045864998E_3,0.0703,0.07353985,0.00000000,0.044055770


protein,best_method_selected,cv_r,cv_r2,standard_deviation_sd_of_cv_r2,predictable_true_if_cv_r2_0_05_and_cv_r_0
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
a1bg,BayesR,0.4701616,0.22115888,0.010844396,TRUE
aamdc,BayesR,0.4888106,0.23930560,0.021103781,TRUE
aarsd1,BayesR,0.2478167,0.06171846,0.009884535,TRUE
abca2,BayesR,0.2816494,0.08212831,0.034040522,TRUE
abhd14b,BayesR,0.2883272,0.08321875,0.006043752,TRUE
abl1,BayesR,0.2690388,0.07250423,0.006600522,TRUE


protein,number_cis_ld_blocks,number_trans_ld_blocks,number_total_ld_blocks,ukbppp_protein_id,cis_pqtls,trans_pqtls,all_pqtls,polygenic_component,total_heritability_th,proportion_of_th_explained_by_primary_cis_p_qtl_component,proportion_of_th_explained_by_primary_trans_p_qtl_component,best_method_selected,cv_r,cv_r2,standard_deviation_sd_of_cv_r2,predictable_true_if_cv_r2_0_05_and_cv_r_0
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
a1bg,466,1,465,A1BG:P04217:OID30771:v1,9.6009252027164205E_2,1.50448618348897E_2,0.11105411386205299,0.1234,0.23445411,0.40950125,0.064169750,BayesR,0.4701616,0.22115888,0.010844396,TRUE
aamdc,391,3,388,AAMDC:Q9H7C9:OID30236:v1,0.22947367197313601,1.32634318179364E_3,0.23080001515492901,0.0774,0.30820002,0.74456087,0.004303514,BayesR,0.4888106,0.23930560,0.021103781,TRUE
aarsd1,391,2,389,AARSD1:Q9BTE6:OID21311:v1,1.1267267571340001E_2,3.4975055187944299E_3,1.47647730901344E_2,0.0523,0.06706477,0.16800575,0.052151157,BayesR,0.2478167,0.06171846,0.009884535,TRUE
abca2,444,2,442,ABCA2:Q9BZC7:OID30146:v1,1.8771451437145899E_3,0,1.8771451437145899E_3,0.0244,0.02627715,0.07143642,0.000000000,BayesR,0.2816494,0.08212831,0.034040522,TRUE
abhd14b,398,2,396,ABHD14B:Q96IU4:OID20921:v1,6.3093345880260401E_2,4.7071662658122399E_3,6.7800512146072703E_2,0.0635,0.13130051,0.48052627,0.035850327,BayesR,0.2883272,0.08321875,0.006043752,TRUE
abl1,403,1,402,ABL1:P00519:OID21280:v1,0,3.2398549045864998E_3,3.2398549045864998E_3,0.0703,0.07353985,0.00000000,0.044055770,BayesR,0.2690388,0.07250423,0.006600522,TRUE


## LD blocks shared between proteins, no additional criteria

In [14]:
# Check how many significant associations there are by protein
# Note: this does not account for CV-r2 criteria
ld_block_by_protein %>%
    filter(signal_fdr0_2 == 1) %>%
    count(protein) %>%
    arrange(desc(n)) %>% 
    head(5)

protein,n
<chr>,<int>
mrpl28,653
sod3,653
saa4,644
clu,642
tnfrsf10b,628


In [13]:
# Get list of all unique LD block regions
ld_blocks <- ld_block_by_protein %>% 
    dplyr::select(region_name) %>% 
    distinct() %>%
    pull(region_name)

# Determine the most commonly appearing LD blocks based on q-value = 0.2, does not account for cv-r2 or cv-4 
ld_block_sharing_q0_2 <- find_ld_block_sharing(ld_block_by_protein, ld_blocks, "signal_fdr0_2") 

ld_block_sharing_q0_2 <- ld_block_sharing_q0_2 %>% 
    count_overlap("region_name")

ld_block_sharing_q0_2 %>% 
    head(10)

ld_block_sharing_q0_2 %>% 
    ungroup() %>%
    summarize(mean = mean(n))

region_name,n
<chr>,<int>
06_31571218_32682664,2298
12_110336719_113263518,2150
19_9238393_11284028,2132
02_26894985_28598777,1955
03_38356116_40221298,1954
12_119754110_122007651,1908
11_70855_1213590,1880
17_7317398_8306425,1868
14_23018665_24905123,1851


mean
<dbl>
775.1644


In [15]:
# LD block for proteins with cvr2 > 0.05 and r > 0 
proteins_r205_r0 <- cv_results %>% 
    filter(cv_r > 0, cv_r2 > 0.05) %>% 
    pull(protein)

ld_block_sharing_r205_r0 <- ld_block_by_protein %>% 
    filter(protein %in% proteins_r205_r0) %>%
    find_ld_block_sharing(ld_blocks, "signal_fdr0_2") %>% 
    count_overlap("region_name")

ld_block_sharing_r205_r0 %>% head(10)

ld_block_sharing_r205_r0 %>% 
    ungroup(region_name) %>%
    summarize(mean = mean(n))

region_name,n
<chr>,<int>
06_31571218_32682664,1839
12_110336719_113263518,1749
19_9238393_11284028,1654
02_26894985_28598777,1600
03_38356116_40221298,1507
12_119754110_122007651,1504
17_7317398_8306425,1496
11_70855_1213590,1490
06_32682664_33236497,1484


mean
<dbl>
613.32


In [16]:
# LD block sharing for proteins with cvr2 greater than 0.05 or r less than 0 
proteins_r205_rbelow0 <- cv_results %>% 
    filter(cv_r < 0 | cv_r2 > 0.05) %>% 
    pull(protein)

ld_block_sharing_r205_rbelow0 <- ld_block_by_protein %>% 
    filter(protein %in% proteins_r205_rbelow0) %>%
    find_ld_block_sharing(ld_blocks, "signal_fdr0_2") %>% 
    count_overlap("region_name")

ld_block_sharing_r205_rbelow0 %>% head(10)

ld_block_sharing_r205_rbelow0 %>% 
    ungroup(region_name) %>%
    summarize(mean = mean(n))

region_name,n
<chr>,<int>
06_31571218_32682664,1889
12_110336719_113263518,1812
19_9238393_11284028,1697
02_26894985_28598777,1648
12_119754110_122007651,1550
03_38356116_40221298,1545
17_7317398_8306425,1545
11_70855_1213590,1529
06_32682664_33236497,1526


mean
<dbl>
633.1439


In [17]:
# LD block sharing for proteins with cvr2 less than 0.05 or r less than 0 
proteins_r2below05_rbelow0 <- cv_results %>% 
    filter(cv_r < 0 | cv_r2 < 0.05) %>% 
    pull(protein) 

ld_block_sharing_r2below05_rbelow0 <- ld_block_by_protein %>% 
    filter(protein %in% proteins_r2below05_rbelow0) %>% 
    find_ld_block_sharing(ld_blocks, "signal_fdr0_2") %>% 
    count_overlap("region_name")

ld_block_sharing_r2below05_rbelow0 %>% head(10)

ld_block_sharing_r2below05_rbelow0 %>% 
    ungroup(region_name) %>%
    summarize(mean = mean(n))

region_name,n
<chr>,<int>
19_9238393_11284028,478
06_31571218_32682664,459
03_38356116_40221298,447
14_23018665_24905123,418
11_47006137_49866050,410
12_119754110_122007651,404
12_110336719_113263518,401
16_65938566_68841363,397
07_149840658_150711505,391


mean
<dbl>
161.8444


In [58]:
# Check if proteins which were predictable for cis-variants only, are predictable for cis and trans variant analysis
cis_predictable_proteins <- cis_only_predict %>%
    filter(avg_r > 0.0 & avg_r2 > 0.05)

not_predictable_cis_trans_only <- cv_results %>% 
    filter(protein %in% cis_predictable_proteins$protein,
          !predictable_true_if_cv_r2_0_05_and_cv_r_0)
not_predictable_cis_trans_only

protein,best_method_selected,cv_r,cv_r2,standard_deviation_sd_of_cv_r2,predictable_true_if_cv_r2_0_05_and_cv_r_0
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>
il12rb1,elastic-net,0.1562986,0.02472278,0.005855977,FALSE
klk1,Lasso,-0.5900986,0.34828796,0.011221493,FALSE
msmb,SuSiE,-0.3954374,0.16643606,0.084489363,FALSE


There were 466 proteins which were shown to be predictable when including cis-variants. After including trans-variants 
in the analysis, there are three proteins that are no longer predictable out of the original 466 proteins. The first protein 
listed is no longer predictable as a result of the reduction in cv_r2. However, the remaining 2 proteins are no longer 
predictable as the cv_r changes signs and becomes negative. 

In [60]:
# Compare cv_r and cv_r2 of proteins predictable by cis but not predictable by cis and trans
cis_predictable_proteins %>%
    filter(protein %in% not_predictable_cis_trans_only$protein) %>%
    select(protein, best_method, avg_r, avg_r2, sd_r2)

protein,best_method,avg_r,avg_r2,sd_r2
<chr>,<chr>,<dbl>,<dbl>,<dbl>
il12rb1,enet_weights,0.2664836,0.07103271,0.002613084
klk1,susie_weights,0.3324904,0.11067333,0.008178648
msmb,bayes_r_weights,0.4289310,0.18407214,0.009025368
